In [116]:
from src.Webscraping import Webscraping_TrailData as TD
from src.Webscraping import Webscraping_TrailReports as TR
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo
import math
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import numpy as np 

In [122]:
hike_urls = pd.read_csv('Olympic_trails_clean.csv')

In [178]:
hike_urls

,Coast,Dogs not allowed,Established campsites,Fall foilage,Good for kids,Lakes,Mountain views,Old growth,Ridges/passes,Rivers,...,lat,long,numReports,number_votes,rating,region,url,which_pass,total_distance,stars
0,0,0,0,0,1,0,0,0,0,0,...,46.8432,-122.9842,0,NaN,NaN,NaN,https://www.wta.org/go-hiking/hikes/scatter-creek,NaN,NaN,NaN
1,0,0,0,0,0,1,1,0,0,0,...,47.4956,-123.2676,270,24.0,3.67 out of 5,Olympic Peninsula -- Hood Canal,https://www.wta.org/go-hiking/hikes/mount-rose,None,6.4,3.67
2,0,0,1,0,1,0,0,1,0,1,...,47.8776,-123.1369,46,3.0,4.67 out of 5,Olympic Peninsula -- Hood Canal,https://www.wta.org/go-hiking/hikes/upper-dung...,Northwest Forest Pass,6.8,4.67
3,1,0,0,0,0,0,0,0,0,1,...,47.2955,-122.7509,1,NaN,NaN,NaN,https://www.wta.org/go-hiking/hikes/maple-holl...,NaN,NaN,NaN
4,1,0,0,0,1,0,1,0,0,0,...,47.9115,-122.5291,17,NaN,NaN,NaN,https://www.wta.org/go-hiking/hikes/point-no-p...,None,NaN,NaN
5,0,1,0,0,1,0,0,0,0,0,...,47.4381,-122.8366,21,11.0,3.09 out of 5,Olympic Peninsula -- Kitsap Peninsula,https://www.wta.org/go-hiking/hikes/mary-e-the...,NaN,3.5,3.09
6,1,1,0,0,1,0,1,1,0,1,...,47.6160,-122.9085,53,14.0,4.07 out of 5,Olympic Peninsula -- Kitsap Peninsula,https://www.wta.org/go-hiking/hikes/guillemot-...,None,2.5,4.07
7,1,0,0,0,1,0,1,1,0,0,...,47.3085,-122.6807,9,2.0,3.50 out of 5,Olympic Peninsula -- Kitsap Peninsula,https://www.wta.org/go-hiking/hikes/kopachuck-...,Discover Pass,1.0,3.50
8,1,1,0,0,1,0,1,0,0,0,...,48.1413,-123.1899,131,21.0,3.48 out of 5,Olympic Peninsula -- Northern Coast,https://www.wta.org/go-hiking/hikes/dungeness-...,Refuge Entrance Pass,11.0,3.48
9,0,0,1,0,0,0,1,1,0,1,...,47.4933,-123.2110,189,26.0,3.92 out of 5,Olympic Peninsula -- Hood Canal,https://www.wta.org/go-hiking/hikes/big-creek,Northwest Forest Pass,4.3,3.92


In [125]:
hike_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 26 columns):
Coast                    360 non-null int64
Dogs not allowed         360 non-null int64
Established campsites    360 non-null int64
Fall foilage             360 non-null int64
Good for kids            360 non-null int64
Lakes                    360 non-null int64
Mountain views           360 non-null int64
Old growth               360 non-null int64
Ridges/passes            360 non-null int64
Rivers                   360 non-null int64
Summits                  360 non-null int64
Waterfalls               360 non-null int64
Wildflowers/Meadows      360 non-null int64
Wildlife                 360 non-null int64
distance                 276 non-null object
elevation_gain           276 non-null float64
highest_point            276 non-null float64
hike_name                360 non-null object
lat                      360 non-null float64
long                     360 non-null float64
numRe

In [131]:
hike_urls['total_distance'] = hike_urls['distance'].apply(lambda x: total_dst(x))

In [157]:
hike_urls['stars'] = hike_urls['rating'].apply(lambda x: stars(str(x)))

In [154]:

def stars(string):
    '''Extracts number of stars'''
    if 'Nan' in string:
        return 'NaN'
    else:
        lst = string.split()
        return float(lst[0])

In [130]:
def total_dst(string):
    """Extracts mileage from string and calculates total mileage depending on if
    description says one-way or roundtrip."""

    if str(string) == 'nan':
        return np.nan
    else:
        miles = []
        try:
            for s in string.split():
                miles.append(float(s))
        except:
            miles.append(string)
        if 'roundtrip' in string:
            return miles[0]
        elif 'one-way' in string:
            return miles[0]*2
        else:
            return miles[0]

In [166]:
def num_votes(string):
    '''Extracts number of votes'''
    if string ==  np.nan:
        return 'NaN'
    else:
        lst = string.strip('(').split()
        return int(lst[0])

In [177]:
hike_urls['number_votes'][0]

nan

In [164]:
hike_urls['number_votes'].apply(lambda x: num_votes(str(x)))

ValueError: invalid literal for int() with base 10: 'nan'

AttributeError: 'list' object has no attribute 'join'

In [179]:
    mc = pymongo.MongoClient()
    db = mc['wta']
    trail_reports = db['trail_reports']
    raw_html = db['html']
    trail_page_raw_html = db['trail_html']

In [180]:
df = pd.DataFrame(list(trail_reports.find()))

In [181]:
df

,Creator,Date,Report,Trail,Trail_condtions,Votes,_id
0,\nrustysailor,"May 06, 2018",Arrived at the trailhead at about 11am to find...,Mount Rose,"\nBeware of:\n snow, trail condit...",3,5af25573acf3d692490fdf8e
1,\nkilpatjw,"May 05, 2018",Got to the trailhead at around 9am and the lot...,Mount Rose,\nBeware of:\n trail conditions\n...,3,5af25573acf3d692490fdf8f
2,\njalaugh,"May 05, 2018",Shortly after the very well marked summit loop...,Mount Rose,"\nBeware of:\n snow, trail condit...",6,5af25573acf3d692490fdf90
3,\nBecuz Its There,"Apr 25, 2018",Made another mad dash up Rose for training. T...,Mount Rose,"\nBeware of:\n snow, trail condit...",7,5af25573acf3d692490fdf91
4,\nCloud Chaser,"Apr 22, 2018","My wife, dog & I headed up Mt Rose this weeken...",Mount Rose,\nBeware of:\n trail conditions\n...,1,5af25573acf3d692490fdf92
5,\nStepbyStep,"Apr 21, 2018",Lots of snow on the loop. Micro spike and pol...,Mount Rose,\nBeware of:\n snow conditions\n ...,4,5af25573acf3d692490fdf94
6,\ntpioletti,"Apr 21, 2018",Parking lot is pretty small but more space on ...,Mount Rose,None,5,5af25573acf3d692490fdf95
7,\nIrene Grinberg,"Apr 20, 2018","Road has some potholes on the way, definitely ...",Mount Rose,"\nBeware of:\n road, snow & trail...",1,5af25573acf3d692490fdf96
8,\nIrene Grinberg,"Apr 20, 2018","Road has some potholes on the way, definitely ...",Mount Rose,"\nBeware of:\n road, snow & trail...",None,5af25573acf3d692490fdf97
9,\nNicole A,"Apr 19, 2018",Started off this sunny morning heading to the ...,Mount Rose,"\nBeware of:\n snow, trail condit...",5,5af25573acf3d692490fdf98


In [77]:
df['Date_type'] = pd.to_datetime(df['Date'])

In [ ]:
df['Creator'] = df['Creator'].apply(lambda x: x.strip('\n'))

In [183]:
len(df['Trail'].unique())

29

In [ ]:
db.trail_reports.count()